# Parte 2:  Sistema RAG - Procesamiento y Chunking
*   **Autor:** Carolina Torres Zapata
*   **Fecha:** 2025-11-24
*   **Contexto:** Una vez adquirido el dato crudo (Capa Bronce), el paso crítico en un sistema RAG es la **segmentación (Chunking)**. Si cortamos el texto arbitrariamente, rompemos las oraciones y el LLM pierde contexto.
*   **Objetivo del Notebook:**
     1.  **Lectura Raw:** Cargar el archivo de texto desde el Volumen de Unity Catalog.
     2.  **Chunking Semántico:** Implementar una lógica que respete los párrafos y oraciones, agrupándolos en bloques de tamaño óptimo (ej. ~1000 caracteres) para la ventana de contexto del LLM.
     3.  **Estructuración:** Convertir la lista de textos en una Tabla Delta (Capa Silver) con identificadores únicos (`chunk_id`) para trazabilidad futura.

## 1. Importar Librerías


In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

## 2. Lectura desde Volumen (Capa Bronce/Raw)
Leemos el archivo directamente desde el almacenamiento gestionado de Databricks

A diferencia de los procesos ETL tradicionales que leen línea por línea (como CSVs), aquí necesitamos el documento entero como una sola unidad para poder analizar sus párrafos.
Usamos `.option("wholetext", True)` para cargar el contenido completo en una sola fila, preservando los saltos de línea (`\n`) que son vitales para identificar la estructura del documento.

In [0]:
ruta_volumen = "/Volumes/dev/bronce/azure_databricks_docs/azure_databricks_intro.txt" 

# 'wholetext' lee todo el archivo en una sola fila de un DataFrame de Spark
df_spark_raw = spark.read.option("wholetext", True).text(ruta_volumen)

print("✅ Texto cargado en Spark DataFrame.")
df_spark_raw.printSchema()

# Extraemos el texto del DataFrame para partirlo
full_text = df_spark_raw.first()[0]

✅ Texto cargado en Spark DataFrame.
root
 |-- value: string (nullable = true)



## 3. Estrategia de Chunking (Preservación de Contexto)
Implementamos una estrategia de **"Ventana Deslizante basada en Párrafos"**.
En lugar de cortar ciegamente cada 1000 caracteres (lo que podría dejar una frase como *"La clave es..."* en un chunk y *"...la seguridad"* en otro), el algoritmo:

1.  Divide el texto por párrafos naturales (`\n\n`).
2.  Agrupa párrafos completos hasta acercarse al límite de tokens/caracteres (1000).
3.  **Regla de Calidad:** Filtra líneas de "navegación web" o pies de página cortos (< 50 chars) que son ruido para el modelo.

In [0]:

# ==========================================
# 1. ESTRATEGIA DE CHUNKING (Lógica Híbrida)
# ==========================================
# Objetivo: Agrupar párrafos completos hasta llegar a un límite de caracteres.
# Beneficio: Evita cortar frases a la mitad y agrupa títulos con su contenido.

# Separación natural por párrafos
paragraphs = [p.strip() for p in full_text.split("\n\n") if p.strip()]

# Inicializar variables
chunks = []
current_chunk = ""
LIMIT_CHARS = 1000  # Límite máximo por chunk

# Iteración para empaquetar párrafos hasta límite
for para in paragraphs:
    # Filtrar párrafos cortos o de sistema
    if len(para) < 50 or "acceso a esta página" in para.lower():
        continue
    
    if len(current_chunk) + len(para) + 2 <= LIMIT_CHARS:
        # Agregar párrafo al chunk actual
        if current_chunk:
            current_chunk += "\n\n" + para
        else:
            current_chunk = para
    else:
        # Guardar chunk actual y empezar uno nuevo
        chunks.append(current_chunk)
        current_chunk = para

# Guardar el último chunk
if current_chunk:
    chunks.append(current_chunk)

print(f"🧩 Se generaron {len(chunks)} chunks optimizados.")
print(f"   Promedio de caracteres por chunk: {sum(len(c) for c in chunks)/len(chunks):.0f}\n")

# Muestra de primeros 3 chunks
for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1} ({len(chunk)} chars): {chunk[:200]}...\n")

🧩 Se generaron 12 chunks optimizados.
   Promedio de caracteres por chunk: 741

Chunk 1 (927 chars): Azure Databricks es una plataforma de análisis unificada y abierta para crear, implementar, compartir y mantener soluciones de datos, análisis e IA de nivel empresarial a escala. Databricks Data Intel...

Chunk 2 (576 chars): Databricks se compromete a la comunidad de código abierto y administra las actualizaciones de integraciones de código abierto con las versiones de Databricks Runtime. Las siguientes tecnologías son pr...

Chunk 3 (504 chars): Data Lakehouse combina almacenes de datos empresariales y lagos de datos para acelerar, simplificar y unificar soluciones de datos empresariales. Los ingenieros de datos, los científicos de datos, los...



## 4. Estructuración y Trazabilidad (Data Modeling)
Para que el sistema RAG funcione, cada fragmento de texto necesita una "cédula de identidad".
Generamos un `chunk_id` único usando `monotonically_increasing_id()`.
*   **Uso posterior:** Cuando el usuario haga una pregunta, el sistema recuperará el ID del chunk más relevante.

In [0]:
# Creamos un DataFrame Spark a partir de la lista de chunks
# Cada chunk será una fila en la columna "chunk_text"
df_chunks = spark.createDataFrame([(c,) for c in chunks], ["chunk_text"])

# Agregamos una columna "chunk_id" con un identificador único para cada fila
# `monotonically_increasing_id()` genera un ID único creciente para cada chunk
df_chunks = df_chunks.withColumn("chunk_id", monotonically_increasing_id())

# Confirmamos que el DataFrame se creó correctamente mostrando las primeras filas
print("✅ DataFrame Spark de chunks creado:")
display(df_chunks)

✅ DataFrame Spark de chunks creado:


chunk_text,chunk_id
"Azure Databricks es una plataforma de análisis unificada y abierta para crear, implementar, compartir y mantener soluciones de datos, análisis e IA de nivel empresarial a escala. Databricks Data Intelligence Platform se integra con el almacenamiento en la nube y la seguridad de su cuenta en la nube, y administra e implementa la infraestructura en la nube para usted. Azure Databricks usa inteligencia artificial generativa con el almacén de lago de datos para comprender la semántica única de los datos. A continuación, optimiza automáticamente el rendimiento y administra la infraestructura para adaptarla a las necesidades de su empresa. El procesamiento de lenguaje natural aprende el idioma de su empresa, por lo que puede buscar y detectar datos haciendo una pregunta en sus propias palabras. La asistencia del lenguaje natural le ayuda a escribir código, solucionar errores y encontrar respuestas en la documentación.",0
"Databricks se compromete a la comunidad de código abierto y administra las actualizaciones de integraciones de código abierto con las versiones de Databricks Runtime. Las siguientes tecnologías son proyectos de código abierto que crearon al principio empleados de Databricks: Los siguientes casos de uso resaltan algunas de las formas en que los clientes usan Azure Databricks para realizar tareas esenciales para procesar, almacenar y analizar los datos que impulsan las decisiones y las funciones empresariales críticas. Creación de un almacén de lago de datos empresarial",1
"Data Lakehouse combina almacenes de datos empresariales y lagos de datos para acelerar, simplificar y unificar soluciones de datos empresariales. Los ingenieros de datos, los científicos de datos, los analistas y los sistemas de producción pueden usar el data lakehouse como origen único de verdad, lo que proporciona acceso a datos coherentes y reduce las complejidades de creación, mantenimiento y sincronización de muchos sistemas de datos distribuidos. Consulte ¿Qué es un almacén de lago de datos? .",2
"Independientemente de si está generando paneles o potenciando aplicaciones de inteligencia artificial, la ingeniería de datos proporciona la red troncal para las empresas centradas en datos al asegurarse de que los datos están disponibles, limpios y almacenados en modelos de datos para una detección y uso eficaces. Azure Databricks combina la eficacia de Apache Spark con Delta y herramientas personalizadas para proporcionar una experiencia ETL inigualable. Use SQL, Python y Scala para componer la lógica ETL y organizar la implementación de trabajos programados con unos pocos clics. Las canalizaciones declarativas de Spark de Lakeflow simplifican aún más ETL mediante la administración inteligente de dependencias entre conjuntos de datos e implementan y escalan automáticamente la infraestructura de producción para garantizar una entrega de datos oportuna y precisa a las especificaciones.",3
"Azure Databricks proporciona herramientas personalizadas para ingesta de datos , incluida Auto Loader , una herramienta eficaz y escalable para cargar datos de forma incremental e idempotente desde el almacenamiento de objetos en la nube y lagos de datos en el centro de lago de datos. Aprendizaje automático, inteligencia artificial y ciencia de datos El aprendizaje automático de Azure Databricks amplía la funcionalidad básica de la plataforma con un conjunto de herramientas adaptadas a las necesidades de los científicos de datos e ingenieros de aprendizaje automático, incluidos MLflow y Databricks Runtime para Machine Learning . Modelos de lenguaje grandes e inteligencia artificial generativa",4
"Databricks Runtime para Machine Learning incluye bibliotecas como Hugging Face Transformers que permiten integrar modelos previamente entrenados existentes u otras bibliotecas de código abierto en el flujo de trabajo. La integración de MLflow de Databricks facilita el uso del servicio de seguimiento de MLflow con canalizaciones, modelos y componentes

## 5. Guardar tabla Delta en Capa Silver
Guardamos el resultado procesado en `dev.silver.rag_chunks` en formato **Delta**.
Esto permite:
1.  **Reutilización:** Si falla el proceso de generación de embeddings, no hay que volver a leer ni procesar el texto.
2.  **Schema Enforcement:** Aseguramos que siempre tengamos las columnas `chunk_text` y `chunk_id`.

In [0]:
target_table = "dev.silver.rag_chunks"

print(f"💾 Guardando tabla Delta en: {target_table} ...")

df_chunks.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("✅ Proceso finalizado exitosamente.")

💾 Guardando tabla Delta en: dev.silver.rag_chunks ...
✅ Proceso finalizado exitosamente.
